In [237]:
import pandas as pd
import glob


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

from pathlib import Path

In [238]:
input_files = str(Path('../4_szures/data/*.csv'))

In [239]:
x = pd.read_csv(glob.glob(input_files)[0]).drop(["Unnamed: 0","Unnamed: 0.1"],axis=1)\
.pipe(lambda df:df.loc[df["sold"]==1]).fillna(0).rename(columns = {"TELJESITMENY LOERO":"LOERO","MEGTETT":"MEGTETT_KM","MEGTETT_2":"MEGTETT_KM_2","BI-XENON":"BI_XENON"})

In [240]:
x = x.loc[x["vegso_ar"]!=0]


In [241]:
y = x["vegso_ar"]
lg10_y = pd.np.log10(y)
x["lg10_AR"] = lg10_y

x = x.drop(['vegso_ar',"lg10_AR"], axis=1)


In [242]:
szukitett = x[["hany_eves","hany_eves_2","MEGTETT_KM","MEGTETT_KM_2","KLÍMA","LÉGZSÁK",\
               "ALUFELNI","ASR","AUTOMATA","BI-XENON","BLUETOOTH","LOERO",\
               "audi","mercedes-benz","bmw","infiniti","jaguar","land_rover","porsche",\
               "lexus","maserati","Hibrid","Elektromos","Dízel","Benzin"]].fillna(0)

KeyError: "['BI-XENON'] not in index"

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(szukitett.dropna(), lg10_y, test_size=0.2, random_state=42)

lreg = LinearRegression()

result = lreg.fit(x_train, y_train)

y_test_pred = result.predict(x_test)

lin_mse = mean_squared_error(y_test, y_test_pred)
lin_rmse = pd.np.sqrt(lin_mse)
lin_rmse

print(result.intercept_)


In [ ]:
pd.DataFrame(szukitett.loc[9]).to_csv("proba.csv")

In [ ]:
import pickle


output_file = str(Path('data/finalized_model.sav'))

# save the model to disk

pickle.dump(result, open(output_file, 'wb'))

In [ ]:
import json

In [ ]:
szukitett["hany_eves"].nunique()

In [ ]:
nuniques = [szukitett[col].nunique() for col in szukitett.columns]

In [ ]:
number_vals = [idx for idx,val in enumerate(nuniques) if val >2]

In [ ]:
auto_marka = ['audi', 'mercedes-benz', 'bmw', 'infiniti',
              'jaguar', 'land_rover', 'porsche', 'lexus', 'maserati']
hajtaslanc = ['Hibrid', 'Elektromos', 'Dízel', 'Benzin']
felszereltseg = szukitett.columns.drop(auto_marka + hajtaslanc + list(szukitett.columns[number_vals]))

In [ ]:
felszereltseg

In [ ]:
details = {"number":list(szukitett.columns[number_vals]),"Dropdown":{"auto_marka":auto_marka,"hajtaslanc":hajtaslanc}\
          ,"boolen":list(felszereltseg),"all":list(szukitett.columns)}

In [ ]:
json_file = json.dumps(details)
f = open("data/dict.json","w")
f.write(json_file)
f.close()

In [ ]:
with open('data/dict.json') as json_file:  
    data_dict = json.load(json_file)

In [ ]:
data_dict

In [ ]:
result.coef_

In [ ]:
len(x_train.columns)

In [ ]:
data["number"]

In [ ]:
with open('data/dict.json') as json_file:  
    data_dict = json.load(json_file)

In [ ]:
numbers  = [val for idx,val in enumerate(data["number"]) if data["number"][idx][:5]!=data["number"][idx-1][:5]]

In [ ]:
list(data["Dropdown"].keys())


In [229]:
data_dict

{'Dropdown': {'auto_marka': ['audi',
   'mercedes-benz',
   'bmw',
   'infiniti',
   'jaguar',
   'land_rover',
   'porsche',
   'lexus',
   'maserati'],
  'hajtaslanc': ['Hibrid', 'Elektromos', 'Dízel', 'Benzin']},
 'all': ['hany_eves',
  'hany_eves_2',
  'MEGTETT_KM',
  'MEGTETT_KM_2',
  'KLÍMA',
  'LÉGZSÁK',
  'ALUFELNI',
  'ASR',
  'AUTOMATA',
  'BI-XENON',
  'BLUETOOTH',
  'LOERO',
  'audi',
  'mercedes-benz',
  'bmw',
  'infiniti',
  'jaguar',
  'land_rover',
  'porsche',
  'lexus',
  'maserati',
  'Hibrid',
  'Elektromos',
  'Dízel',
  'Benzin'],
 'boolen': ['KLÍMA',
  'LÉGZSÁK',
  'ALUFELNI',
  'ASR',
  'AUTOMATA',
  'BI-XENON',
  'BLUETOOTH'],
 'number': ['hany_eves', 'hany_eves_2', 'MEGTETT_KM', 'MEGTETT_KM_2', 'LOERO']}

In [236]:
{str(x):[x] for x in col_names}

{'ALUFELNI': ['ALUFELNI'],
 'ASR': ['ASR'],
 'AUTOMATA': ['AUTOMATA'],
 'BI-XENON': ['BI-XENON'],
 'BLUETOOTH': ['BLUETOOTH'],
 'KLÍMA': ['KLÍMA'],
 'LOERO': ['LOERO'],
 'LÉGZSÁK': ['LÉGZSÁK'],
 'MEGTETT_KM': ['MEGTETT_KM'],
 'auto_marka': ['auto_marka'],
 'hajtaslanc': ['hajtaslanc'],
 'hany_eves': ['hany_eves']}

In [233]:
minden_szar = list(data_dict["Dropdown"].keys()) + data_dict["boolen"] + data_dict["number"]

In [235]:
numbers = [val for idx,val in enumerate(data_dict["number"]) if data_dict["number"][idx][:5]!=data_dict["number"][idx-1][:5]]

col_names = list(data_dict["Dropdown"].keys()) + data_dict["boolen"] + numbers

In [ ]:
[Input('hany_eves', 'value'), Input('MEGTETT_KM', 'value'),Input('LOERO', 'value'), Input('KLÍMA', 'value'), Input('LÉGZSÁK', 'value'),
Input('ALUFELNI', 'value'), Input('ASR', 'value'), Input('AUTOMATA', 'value'), Input('BI-XENON', 'value'), Input('BLUETOOTH', 'value'), 
Input('auto_marka', 'value'), Input('hajtaslanc', 'value')])

In [227]:
for key in list(data_dict["Dropdown"].keys()):
    
    print(key)

auto_marka
hajtaslanc


In [243]:
data_dict

{'Dropdown': {'auto_marka': ['audi',
   'mercedes-benz',
   'bmw',
   'infiniti',
   'jaguar',
   'land_rover',
   'porsche',
   'lexus',
   'maserati'],
  'hajtaslanc': ['Hibrid', 'Elektromos', 'Dízel', 'Benzin']},
 'all': ['hany_eves',
  'hany_eves_2',
  'MEGTETT_KM',
  'MEGTETT_KM_2',
  'KLÍMA',
  'LÉGZSÁK',
  'ALUFELNI',
  'ASR',
  'AUTOMATA',
  'BI-XENON',
  'BLUETOOTH',
  'LOERO',
  'audi',
  'mercedes-benz',
  'bmw',
  'infiniti',
  'jaguar',
  'land_rover',
  'porsche',
  'lexus',
  'maserati',
  'Hibrid',
  'Elektromos',
  'Dízel',
  'Benzin'],
 'boolen': ['KLÍMA',
  'LÉGZSÁK',
  'ALUFELNI',
  'ASR',
  'AUTOMATA',
  'BI-XENON',
  'BLUETOOTH'],
 'number': ['hany_eves', 'hany_eves_2', 'MEGTETT_KM', 'MEGTETT_KM_2', 'LOERO']}

In [ ]:

@app.callback(Output('titkos', 'children'),\
[Input('hany_eves', 'value')])
def titok(hany_eves, MEGTETT_KM, LOERO, KLÍMA, LÉGZSÁK, ALUFELNI, ASR, AUTOMATA, BI_XENON, BLUETOOTH, hajtaslanc,auto_marka):
    d={'hany_eves': [hany_eves], 'MEGTETT': [MEGTETT_KM], 'LOERO': [LOERO],"KLÍMA":[KLÍMA], 'LÉGZSÁK': [LÉGZSÁK], 'ALUFELNI': [ALUFELNI], 'ASR': [ASR],
    'AUTOMATA': [AUTOMATA], 'BI_XENON': [BI_XENON], 'BLUETOOTH': [BLUETOOTH], 'Márka': [auto_marka],
    'hajtaslanc': [hajtaslanc]}


    df = pd.DataFrame(data=d)


    df = df.join(pd.get_dummies(df["Márka"]))
    df = df.drop(["Márka"], axis = 1)

    df = df.join(pd.get_dummies(df["Üzemanyag"]))
    df = df.drop(["Üzemanyag"], axis = 1)

    df = df.applymap(int)

   # df["hany_eves_2"] = df["hany_eves"] ** 2
    #df["MEGTETT_2"] = df["MEGTETT"] ** 2

    #df.to_csv("proba_2.csv")

    print(df)

    df_final = pd.merge(pd.DataFrame(0,index=range(1),columns=col_names), df, how = "right",on = list(df.columns)).fillna(0)

    auto_erteke = 10 ** loaded_model.predict(df_final)[0]


    print(auto_erteke)
    return "az általad megadott auto értéke: " + str(int(auto_erteke)) + " Ft"


In [259]:
list(szukitett.columns)

['hany_eves',
 'hany_eves_2',
 'MEGTETT_KM',
 'MEGTETT_KM_2',
 'KLÍMA',
 'LÉGZSÁK',
 'ALUFELNI',
 'ASR',
 'AUTOMATA',
 'BI-XENON',
 'BLUETOOTH',
 'LOERO',
 'audi',
 'mercedes-benz',
 'bmw',
 'infiniti',
 'jaguar',
 'land_rover',
 'porsche',
 'lexus',
 'maserati',
 'Hibrid',
 'Elektromos',
 'Dízel',
 'Benzin']

In [260]:
col_names


['hany_eves',
 'MEGTETT_KM',
 'LOERO',
 'KLÍMA',
 'LÉGZSÁK',
 'ALUFELNI',
 'ASR',
 'AUTOMATA',
 'BI-XENON',
 'BLUETOOTH',
 'auto_marka',
 'hajtaslanc']

In [244]:
def fing(*args):
    return args

In [245]:
fing(10,20,30)

(10, 20, 30)

In [246]:
arglist = ['foo', 'fűű', 'ajj']

In [251]:
fing(*arglist)

('foo', 'fűű', 'ajj')

In [254]:
col_names = numbers + data_dict["boolen"]+ list(data_dict["Dropdown"].keys())


In [255]:
col_names

['hany_eves',
 'MEGTETT_KM',
 'LOERO',
 'KLÍMA',
 'LÉGZSÁK',
 'ALUFELNI',
 'ASR',
 'AUTOMATA',
 'BI-XENON',
 'BLUETOOTH',
 'auto_marka',
 'hajtaslanc']